In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/Spring2020/CIS700-001/project_1/to_submit'
!pip install sentencepiece

/content/drive/My Drive/Spring2020/CIS700-001/project_1/to_submit
     |████████████████████████████████| 1.0MB 25.6MB/s 


In [3]:
from DMN_Plus_RACE2 import *

In [0]:
HOMEWORK_FOLDER = '/content/drive/My Drive/Spring2020/CIS700-001/project_1/to_submit/'
CHECKPOINT_PATH = HOMEWORK_FOLDER + 'dmn_race_checkpt.pth'
dset_dict, vocab_dict = {}, {}
# for task_id in range(1, 21):
#     dset_dict[task_id] = BabiDataset(task_id)
#     vocab_dict[task_id] = len(dset_dict[task_id].QA.VOCAB)
for run in range(1):
    # for task_id in range(1, 21):
      dset = RACEDataset(mode='train')
      vocab_size = len(dset.QA.VOCAB)
      hidden_size = 80
      print('debug model size: ', vocab_size, hidden_size)
      model = DMNPlus(hidden_size, vocab_size, num_hop=3, qa=dset.QA)
      model # .cuda()
      torch.save(model.state_dict(), CHECKPOINT_PATH)
      early_stopping_cnt = 0
      early_stopping_flag = False
      best_acc = 0
      optim = torch.optim.Adam(model.parameters())

      for epoch in range(256):
          dset.set_mode('train')
          train_loader = DataLoader(dset, batch_size=2, shuffle=True, collate_fn=pad_collate)

          model.train()
          if not early_stopping_flag:
              total_acc = 0
              cnt = 0
              for batch_idx, data in enumerate(train_loader):
                  optim.zero_grad()
                  contexts, questions, answers, options = data
                  # print('debug msg data shape: ', contexts.shape, questions.shape, answers.shape)
                  batch_size = contexts.size()[0]
                  contexts = contexts.long() # .cuda()
                  questions = questions.long() # .cuda()
                  answers = answers # .cuda()

                  loss, acc = model.get_loss(contexts, questions, answers, options)
                  loss.backward()
                  total_acc += acc * batch_size
                  cnt += batch_size

                  if batch_idx % 20 == 0:
                      print(f'Epoch {epoch}] [Training] loss : {loss.item(): {10}.{8}}, '
                            f'acc : {total_acc / cnt: {5}.{4}}, batch_idx : {batch_idx}')
                  optim.step()
                  if batch_idx % 100 == 0:
                      torch.save(model.state_dict(), CHECKPOINT_PATH)

              dset.set_mode('valid')
              valid_loader = DataLoader(dset, batch_size=2, shuffle=False, collate_fn=pad_collate)

              model.eval()
              total_acc = 0
              cnt = 0
              for batch_idx, data in enumerate(valid_loader):
                  contexts, questions, answers, options = data
                  batch_size = contexts.size()[0]
                  contexts = contexts.long() # .cuda()
                  questions = questions.long() # .cuda()
                  answers = answers # .cuda()

                  _, acc = model.get_loss(contexts, questions, answers, options)
                  total_acc += acc * batch_size
                  cnt += batch_size

              total_acc = total_acc / cnt
              if total_acc > best_acc:
                  best_acc = total_acc
                  best_state = model.state_dict()
                  early_stopping_cnt = 0
              else:
                  early_stopping_cnt += 1
                  if early_stopping_cnt > 20:
                      early_stopping_flag = True

              print(f'[Run {run}, Epoch {epoch}] [Validate] Accuracy : {total_acc: {5}.{4}}')
              with open(HOMEWORK_FOLDER + 'log_b128.txt', 'a') as fp:
                  fp.write(
                      f'[Run {run}, Epoch {epoch}] [Validate] Accuracy : {total_acc: {5}.{4}}' + '\n')
              if total_acc == 1.0:
                  break
          else:
              print(
                  f'[Run {run}] Early Stopping at Epoch {epoch}, Valid Accuracy : {best_acc: {5}.{4}}')
              break

      dset.set_mode('test')
      test_loader = DataLoader(dset, batch_size=2, shuffle=False, collate_fn=pad_collate)
      test_acc = 0
      cnt = 0

      for batch_idx, data in enumerate(test_loader):
          contexts, questions, answers, options = data
          batch_size = contexts.size()[0]
          contexts = contexts.long() # .cuda()
          questions = questions.long() # .cuda()
          answers = answers # .cuda()

          model.load_state_dict(best_state)
          _, acc = model.get_loss(contexts, questions, answers, options)
          test_acc += acc * batch_size
          cnt += batch_size
      print(f'[Run {run}, Epoch {epoch}] [Test] Accuracy : {test_acc / cnt: {5}.{4}}')
      os.makedirs('models', exist_ok=True)
      with open(f'models/epoch{epoch}_run{run}_acc{test_acc / cnt}.pth', 'wb') as fp:
          torch.save(model.state_dict(), fp)
      with open(HOMEWORK_FOLDER + 'log_b128.txt', 'a') as fp:
          fp.write(f'[Run {run}, Epoch {epoch}] [Test] Accuracy : {total_acc: {5}.{4}}' + '\n')
